# Classification Iris avec un K-NN

Quelques packages importants

In [ ]:
import collections
import matplotlib
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn import datasets, neighbors
import pandas
import time

%matplotlib notebook
plt.rcParams['figure.figsize'] = [10, 8]

Ici, on utilise le dataset Iris de sklearn. Il est formé de 150 fleurs dont chacune possède 4 caractéristiques différentes (sepal length en cm, sepal width en cm, petal length en cm et petal width en cm) et fait partie d'une classe distincte 'setosa', 'versicolor' ou 'virginica'.

In [ ]:
iris = datasets.load_iris(as_frame=True)
X = iris.data
y = iris.target

print(f"{X.shape = }, {y.shape = }")
print(f"{np.unique(y) = }")

Ensuite, on définit les hyperparamètres de notre K-NN.

In [ ]:
data_and_target = pandas.concat([iris.data, iris.target], axis=1)

# Cette ligne crée une liste contenant toutes les paires possibles
# entre les 4 mesures.
# Par exemple : [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]
pairs = [(i, j) for i in range(4) for j in range(i+1, 4)]

# Utilisons cette liste de paires pour afficher les données, deux
# mesures à la fois.
# On crée une figure à plusieurs sous-graphes.
fig, subfigs = plt.subplots(2, 3, tight_layout=True)
for (f1, f2), subfig in zip(pairs, subfigs.reshape(-1)):
    # Affichez les données en utilisant f1 et f2 comme mesures
    legend_label = ["Iris Setosa", "Iris Versicolore", "Iris Virginia"]
    colors = ['#FF0000', '#f2faf5', '#0000FF']
    
    # Classe 0 - Iris Setosa
    subfig.scatter(x = data_and_target[data_and_target['target'] == 0].iloc[:, [f1]].values, 
                   y = data_and_target[data_and_target['target'] == 0].iloc[:, [f2]].values,
                   c = colors[0],
                   edgecolor = 'k',
                   label = legend_label[0]
                  )
    
    # Classe 1 - Iris Versicolore
    subfig.scatter(x = data_and_target[data_and_target['target'] == 1].iloc[:, [f1]].values, 
                   y = data_and_target[data_and_target['target'] == 1].iloc[:, [f2]].values,
                   c = colors[1],
                   edgecolor = 'k',
                   label = legend_label[1]
                  )
    
    # Classe 2 - Iris Virginia
    subfig.scatter(x = data_and_target[data_and_target['target'] == 2].iloc[:, [f1]].values, 
                   y = data_and_target[data_and_target['target'] == 2].iloc[:, [f2]].values, 
                   c = colors[2],
                   edgecolor = 'k',
                   label = legend_label[2]
                  )
    
    subfig.legend(fontsize=7)
    
    axis_label_list = ['Sepal length [cm]',
                       'Sepal width [cm]',
                       'Petal length [cm]',
                       'Petal width [cm]'
                      ]
    
    subfig.set_xlabel(xlabel = axis_label_list[f1])
    subfig.set_ylabel(ylabel = axis_label_list[f2])


In [ ]:
n_neighbors = 15
weights = "uniform"

Maintenant, on crée notre K-NN et on le fit (entraîne) sur les données d'Iris. À Noté que l'entraînement du K-NN est seulement une mise en mémoire des données, car le modèle de K-NN fait ses prédictions directement en comparant une nouvelle donnée avec celles du dataset.

In [ ]:
clf = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)

Ici, on crée une petite fonction servant à afficher les frontières de décisions sur les caractéristiques des sépales. Il est à noter que le code pourrait facilement être modifié pour afficher les autres caractéristiques.

In [ ]:
# Dictionnaire pour enregistrer les erreurs selon les 
# classifieurs
erreurs = {}

# Cette ligne crée une liste contenant toutes les paires possibles
# entre les 4 mesures.
# Par exemple : [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]
pairs = [(i, j) for i in range(4) for j in range(i+1, 4)]

# Reprenons les paires de mesures
fig, subfigs = plt.subplots(2, 3, tight_layout=True)
t1 = time.time()
for (f1, f2), subfig in zip(pairs, subfigs.reshape(-1)):
    f1_name = iris.feature_names[f1]
    f2_name = iris.feature_names[f2]
    
    # Créez ici un sous-dataset contenant seulement les
    # mesures désignées par f1 et f2
    subdataset = pandas.concat([iris.data[f1_name], iris.data[f2_name]], axis=1)
    X = subdataset.values
    R = iris.target.values

    # Créez ici une grille permettant d'afficher les régions de
    # décision pour chaque classifieur
    h = .015
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    # Entraînez le classifieur
    clf.fit(X, R)

    # Obtenez et affichez son erreur (1 - accuracy)
    err = 1 - clf.score(X, R)

    # Ajout de l'erreur pour affichage
    erreurs[f'{f1_name} {f2_name}'] = err

    # Utilisez la grille que vous avez créée plus haut
    # pour afficher les régions de décision, de même
    # que les points colorés selon leur vraie classe
    Y = clf.predict(np.c_[xx.ravel(), yy.ravel()])

    cm = plt.cm.RdBu
    colors = ['#FF0000', '#f2faf5', '#0000FF']
    legend_labels = ["Iris Setosa", "Iris Versicolore", "Iris Virginia"]

    Y = Y.reshape(xx.shape)
    subfig.contourf(xx, yy, Y, cmap=cm, alpha=0.8)
    scatter = subfig.scatter(X[:, 0], X[:, 1], c=np.array(colors)[R].tolist(), edgecolor = 'k', linewidths = 1.2)
    subfig.set_xlim(xx.min(), xx.max())
    subfig.set_ylim(yy.min(), yy.max())

    # Identification des axes et des méthodes
    red_patch = matplotlib.lines.Line2D([0], [0], marker='o', linestyle = 'None', markerfacecolor='red', 
                                        markersize=5, markeredgecolor='k', label=legend_labels[0])
    white_patch = matplotlib.lines.Line2D([0], [0], marker='o', linestyle = 'None', markerfacecolor='white', 
                                          markersize=5, markeredgecolor='k', label=legend_labels[1])
    blue_patch = matplotlib.lines.Line2D([0], [0], marker='o', linestyle = 'None', markerfacecolor='blue', 
                                         markersize=5, markeredgecolor='k', label=legend_labels[2])

    subfig.legend(handles=[red_patch, white_patch, blue_patch], fontsize=7)
    subfig.set_xlabel(f1_name)
    subfig.set_ylabel(f2_name)


# Affichage des erreurs
df = pandas.DataFrame(erreurs, index=["KNN"])
display(df)